<a href="https://colab.research.google.com/github/antoniivanov/vdk-demo/blob/main/data_sources.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install quickstart-vdk vdk-notebook vdk-ipython vdk-duckdb

In [ ]:
!pip install -U --index-url https://test.pypi.org/simple vdk-data-sources vdk-singer vdk-huggingface vdk-data-source-git vdk-ipython

In [ ]:
%env DB_DEFAULT_TYPE=sqlite
%env INGEST_METHOD_DEFAULT=sqlite
%env INGESTER_WAIT_TO_FINISH_AFTER_EVERY_SEND=true

In [2]:
%load_ext vdk.plugin.ipython
%reload_VDK

In [ ]:
job_input = VDK.get_initialized_job_input()


In [ ]:
!vdk data-sources --list

In [ ]:
!vdk singer --list-taps

In [ ]:
!pip install tap-rest-api-msdk

In [4]:

from vdk.plugin.data_sources.mapping.data_flow import DataFlowInput
from vdk.plugin.data_sources.mapping.definitions import DestinationDefinition
from vdk.plugin.data_sources.mapping.definitions import SourceDefinition

config = dict(tap_name="tap-rest-api-msdk",
              tap_config={
                  "api_url": "https://jsonplaceholder.typicode.com",
                  "streams": [
                      {
                          "name": "users",
                          "path": "/users",
                          "records_path": "$.[*]",
                          "num_inference_records": 200,
                      }
                  ],
              },
              tap_auto_discover_schema=True)

source=SourceDefinition(id="users", name="singer-tap", config=config)
sqlite_destination=DestinationDefinition(id="sqlite", method="sqlite")



In [ ]:
with DataFlowInput(job_input) as flow_input:
  flow_input.start(source, destination)

In [ ]:
%%ingest

[sources.users]
name = "singer-tap"
config.tap_name = "tap-rest-api-msdk"

[sources.users.config.tap_config]
api_url = "https://jsonplaceholder.typicode.com"

[[sources.users.config.tap_config.streams]]
name = "users"
path = "/users"
records_path = "$.[*]"
num_inference_records = 200

[destinations.sqlite]
method="sqlite"

[[flows]]
from="users"
to="sqlite"

In [ ]:
%%vdksql
select * from users